In [1]:
import pandas as pd
import csv

In [2]:
r = pd.read_csv( 'goodbooks/ratings.csv' )

In [3]:
test=r.loc[pd.to_numeric(r['rating'])>3]
liked_books=test.groupby('user_id')['book_id'].apply(set)#.reset_index(name="liked")
liked_books.to_json('liked_books.json')

In [4]:
id=1

In [5]:
my_books=test.groupby('user_id')

In [6]:
my_books.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
5,2,26,4
...,...,...,...
5976098,49802,8694,4
5976099,49802,9747,4
5976100,49802,9892,4
5976101,49802,9488,4


In [7]:
books_titles = pd.read_json("books_titles.json")

In [10]:
books_titles.head()

,book_id,title,ratings,cover_image,mod_title
0,1,The Hunger Games,22806,https://images.gr-assets.com/books/1447303603m...,thehungergames
1,2,Harry Potter and the Philosopher's Stone,21850,https://images.gr-assets.com/books/1474154022m...,harrypotterandthephilosophersstone
2,3,Twilight,16931,https://images.gr-assets.com/books/1361039443m...,twilight
3,4,To Kill a Mockingbird,19088,https://images.gr-assets.com/books/1361975680m...,tokillamockingbird
4,5,The Great Gatsby,16604,https://images.gr-assets.com/books/1490528560m...,thegreatgatsby


In [48]:
import numpy as np
liked_book_id=liked_books[id]
#liked_book_id=pd.DataFrame(liked_book_id)
#liked_book_id.head()
#liked_book_id=liked_book_id.astype(str)
books_titles['book_id']=books_titles['book_id'].astype(str)

In [52]:
my_books=books_titles[pd.to_numeric(books_titles['book_id']).isin(liked_book_id)].copy()

In [54]:
my_books.head()

,book_id,title,ratings,cover_image,mod_title
3,4,To Kill a Mockingbird,19088,https://images.gr-assets.com/books/1361975680m...,tokillamockingbird
9,10,Pride and Prejudice,13445,https://images.gr-assets.com/books/1320399351m...,prideandprejudice
10,11,The Kite Runner,12698,https://images.gr-assets.com/books/1484565687m...,thekiterunner
12,13,Nineteen Eighty-Four,14693,https://images.gr-assets.com/books/1348990566m...,nineteeneightyfour
30,31,The Help,12727,https://images.gr-assets.com/books/1346100365m...,thehelp


In [83]:
my_books["book_id"] = my_books["book_id"].astype(str)
user=[id]*len(my_books)
my_books.insert(0,'user_id',user)

In [84]:
my_books.head()

,user_id,book_id,title,ratings,cover_image,mod_title
3,1,4,To Kill a Mockingbird,19088,https://images.gr-assets.com/books/1361975680m...,tokillamockingbird
9,1,10,Pride and Prejudice,13445,https://images.gr-assets.com/books/1320399351m...,prideandprejudice
10,1,11,The Kite Runner,12698,https://images.gr-assets.com/books/1484565687m...,thekiterunner
12,1,13,Nineteen Eighty-Four,14693,https://images.gr-assets.com/books/1348990566m...,nineteeneightyfour
30,1,31,The Help,12727,https://images.gr-assets.com/books/1346100365m...,thehelp


In [56]:
book_set = set(my_books["book_id"])

In [61]:
overlap_users = {}

with open( 'goodbooks/ratings.csv', 'r') as f:
    csvReader = csv.DictReader(f)

    # convert each csv row into python dict
    book_titles=[]
    for l in csvReader:
        if l['book_id'] in book_set:
            if l['user_id'] not in overlap_users:
                overlap_users[l['user_id']] = 1
            else:
                overlap_users[l['user_id']] += 1


In [63]:
len(overlap_users)



47534

In [64]:
filtered_overlap_users = set([k for k in overlap_users if overlap_users[k] > my_books.shape[0]/5])
len(filtered_overlap_users)

5870

In [68]:
interactions_list = []

with open('goodbooks/ratings.csv', 'r') as f:
    
    csvReader = csv.DictReader(f)

    # convert each csv row into python dict
    for l in csvReader:
        if l['user_id'] in filtered_overlap_users:
        
            interactions_list.append([l['user_id'],l['book_id'],l['rating']])

In [69]:
len(interactions_list)

774756

In [70]:
interactions_list[0]

['1', '258', '5']

In [87]:
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "ratings"])
interactions = pd.concat([my_books[['user_id','book_id', "ratings"]], interactions])
interactions

,user_id,book_id,ratings
3,1,4,19088
9,1,10,13445
10,1,11,12698
12,1,13,14693
30,1,31,12727
...,...,...,...
774751,50480,1539,3
774752,50480,2772,4
774753,51968,2003,4
774754,53358,253,5


In [89]:
interactions["book_id"] = interactions["book_id"].astype(str)
interactions["user_id"] = interactions["user_id"].astype(str)
interactions["ratings"] = pd.to_numeric(interactions["ratings"])

In [90]:
interactions["user_index"] = interactions["user_id"].astype("category").cat.codes
interactions["book_index"] = interactions["book_id"].astype("category").cat.codes

In [91]:
from scipy.sparse import coo_matrix

ratings_mat_coo = coo_matrix((interactions["ratings"], (interactions["user_index"], interactions["book_index"])))
ratings_mat_coo.shape

(5870, 9373)

In [93]:
ratings_mat = ratings_mat_coo.tocsr()
interactions[interactions["user_id"] == str(id)]

,user_id,book_id,ratings,user_index,book_index
3,1,4,19088,0,3304
9,1,10,13445,0,1
10,1,11,12698,0,113
12,1,13,14693,0,335
30,1,31,12727,0,2328
...,...,...,...,...,...
753401,1,142,4,0,469
753402,1,642,4,0,5851
753403,1,901,4,0,8446
753404,1,212,3,0,1244


In [94]:
my_index = 0

In [95]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index,:], ratings_mat).flatten()

In [96]:
similarity[0]

0.9999999999999994

In [97]:
import numpy as np

indices = np.argpartition(similarity, -15)[-15:]

In [98]:
indices

array([4170, 1448, 3795, 2935,  228, 3817,   86,  279, 4002, 2692, 1559,
       1323, 3484,  379,    0], dtype=int64)

In [99]:
similar_users = interactions[interactions["user_index"].isin(indices)].copy()

In [100]:
similar_users = similar_users[similar_users["user_id"]!=str(id)]
similar_users

,user_id,book_id,ratings,user_index,book_index
20639,3194,4,5,2935,3304
20640,3194,8,4,2935,7436
20641,3194,5,4,2935,4380
20642,3194,26,3,2935,1774
20643,3194,11,5,2935,113
...,...,...,...,...,...
767153,19142,54,4,1323,4804
767154,19142,551,4,1323,4919
767155,19142,1011,4,1323,16
769207,20893,61,4,1559,5525


In [102]:
book_recs = similar_users.groupby("book_id").ratings.agg(['count', 'mean'])

In [103]:
book_recs

,count,mean
book_id,,
1,8,3.875000
10,14,4.357143
100,13,3.769231
101,5,3.000000
1011,4,3.500000
...,...,...
985,2,4.000000
9860,1,3.000000
99,1,3.000000


In [104]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [105]:
book_recs = book_recs.merge(books_titles, how="inner", on="book_id")

In [106]:
book_recs

,book_id,count,mean,title,ratings,cover_image,mod_title
0,1,8,3.875000,The Hunger Games,22806,https://images.gr-assets.com/books/1447303603m...,thehungergames
1,10,14,4.357143,Pride and Prejudice,13445,https://images.gr-assets.com/books/1320399351m...,prideandprejudice
2,100,13,3.769231,The Poisonwood Bible,6343,https://images.gr-assets.com/books/1412242487m...,thepoisonwoodbible
3,101,5,3.000000,Me Talk Pretty One Day,5199,https://s.gr-assets.com/assets/nophoto/book/11...,metalkprettyoneday
4,1011,4,3.500000,People of the Book,1493,https://s.gr-assets.com/assets/nophoto/book/11...,peopleofthebook
...,...,...,...,...,...,...,...
683,985,2,4.000000,The Big Short: Inside the Doomsday Machine,855,https://images.gr-assets.com/books/1453470736m...,thebigshortinsidethedoomsdaymachine
684,9860,1,3.000000,The Last Picture Show,158,https://images.gr-assets.com/books/1344275927m...,thelastpictureshow
685,99,1,3.000000,Fifty Shades Darker,4452,https://images.gr-assets.com/books/1358266080m...,fiftyshadesdarker
686,992,1,5.000000,The Twilight Saga,932,https://images.gr-assets.com/books/1327930511m...,thetwilightsaga


In [107]:
book_recs["adjusted_count"] = book_recs["count"] * (book_recs["count"] / book_recs["ratings"])
book_recs["score"] = book_recs["mean"] * book_recs["adjusted_count"]
book_recs = book_recs[~book_recs["book_id"].isin(my_books["book_id"])]
my_books["mod_title"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()
my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)
book_recs = book_recs[~book_recs["mod_title"].isin(my_books["mod_title"])]
book_recs = book_recs[book_recs["mean"] >=4]
book_recs = book_recs[book_recs["count"]>2]
top_recs = book_recs.sort_values("mean", ascending=False)

In [112]:
top_recs=top_recs[:50]
top_recs.to_csv('top_recs.csv')

In [110]:
def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

top_recs.style.format({ 'cover_image': show_image})


,book_id,count,mean,title,ratings,cover_image,mod_title,adjusted_count,score
155,178,3,5.000000,The Bell Jar,5178,,thebelljar,0.001738,0.008691
334,325,4,5.000000,A Prayer for Owen Meany,3868,,aprayerforowenmeany,0.004137,0.020683
643,87,4,5.000000,Un di Velt Hot Geshvign,6466,,undivelthotgeshvign,0.002474,0.012372
526,620,3,5.000000,The Mists of Avalon,2380,,themistsofavalon,0.003782,0.018908
442,47,7,4.714286,The Book Thief,10394,,thebookthief,0.004714,0.022224
386,39,3,4.666667,A Game of Thrones,10692,,agameofthrones,0.000842,0.003928
404,413,3,4.666667,Les Trois Mousquetaires,3062,,lestroismousquetaires,0.002939,0.013717
121,155,3,4.666667,The Two Towers,6478,,thetwotowers,0.001389,0.006483
125,158,3,4.666667,Charlie and the Chocolate Factory,6034,,charlieandthechocolatefactory,0.001492,0.006961
580,73,3,4.666667,The Host,6313,,thehost,0.001426,0.006653
